In [1]:
import time
import numpy as np
import pandas as pd
import random
import torch as t
import copy
from fastcore.foundation import patch
from nixtla.data.ts_dataset import TimeSeriesDataset
from nixtla.data.ts_loader_general import TimeSeriesLoader
from nixtla.data.ts_loader_pinche import TimeSeriesLoader as TimeSeriesLoaderPinche
from nixtla.data.ts_loader import TimeSeriesLoader as TimeSeriesLoaderFast

In [2]:
from nixtla.data.datasets.tourism import Tourism, TourismInfo

In [3]:
tourism_dataset = Tourism.load(directory='data', group=TourismInfo.groups[0])

Processing dataframes ...
Creating ts tensor ...


In [4]:
tourism_dataset.ts_tensor.shape

(518, 3, 47)

In [5]:
ts_loader = TimeSeriesLoader(ts_dataset=tourism_dataset,
                             model='nbeats',
                             offset=0,
                             window_sampling_limit=200*4, 
                             input_size=3*4,
                             output_size=4,
                             idx_to_sample_freq=1,
                             batch_size= 2048,
                             n_series_per_batch=32)

ts_loader_pinche = TimeSeriesLoaderPinche(ts_dataset=tourism_dataset,
                                        model='nbeats',
                                        offset=0,
                                        window_sampling_limit=200*4, 
                                        input_size=3*4,
                                        output_size=4,
                                        idx_to_sample_freq=1,
                                        batch_size= 2048,
                                        train_loader=True)

ts_loader_fast = TimeSeriesLoaderFast(ts_dataset=tourism_dataset,
                                        model='nbeats',
                                        offset=0,
                                        window_sampling_limit=200*4, 
                                        input_size=3*4,
                                        output_size=4,
                                        idx_to_sample_freq=1,
                                        batch_size= 2048)           

In [6]:
dataloader = iter(ts_loader)
batch = next(dataloader)
start = time.time()
insample_y = batch['insample_y']
insample_x_t = batch['insample_x_t']
insample_mask = batch['insample_mask']
outsample_x_t = batch['outsample_x_t']
outsample_y = batch['outsample_y']
outsample_mask = batch['outsample_mask']
time.time()-start

0.0002040863037109375

In [7]:
insample_y.shape

torch.Size([2048, 12])

In [8]:
dataloader = iter(ts_loader_pinche)
batch = next(dataloader)
start = time.time()
insample_y = batch['insample_y']
insample_x_t = batch['insample_x_t']
insample_mask = batch['insample_mask']
outsample_x_t = batch['outsample_x_t']
outsample_y = batch['outsample_y']
outsample_mask = batch['outsample_mask']
time.time()-start

0.00017714500427246094

In [9]:
insample_y.shape

(2048, 12)

In [10]:
dataloader = iter(ts_loader_fast)
batch = next(dataloader)
start = time.time()
insample_y = batch['insample_y']
insample_x_t = batch['insample_x_t']
insample_mask = batch['insample_mask']
outsample_x_t = batch['outsample_x_t']
outsample_y = batch['outsample_y']
outsample_mask = batch['outsample_mask']
time.time()-start

0.0002162456512451172

In [11]:
insample_y.shape

torch.Size([2048, 12])

In [12]:
ts_loader_fast.ts_windows

tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        ...,

        [[3.1790e+00, 3.8999e+00, 3.3920e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0

In [13]:
ts_loader_fast.ts_windows[20000:20010]

tensor([[[7.3340e+03, 9.3160e+03, 1.0235e+04, 1.0301e+04, 9.9970e+03,
          1.0151e+04, 1.1513e+04, 1.4826e+04, 1.6453e+04, 1.7192e+04,
          1.7708e+04, 1.6348e+04, 1.8758e+04, 2.1036e+04, 2.4312e+04,
          3.1589e+04],
         [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
          1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
          1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
          1.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
          1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
          1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
          1.0000e+00]],

        [[7.4900e+03, 7.8900e+03, 1.0304e+04, 1.4003e+04, 1.4700e+04,
          1.3823e+04, 1.3206e+04, 1.7696e+04, 2.2609e+04, 2.5896e+04,
          2.2330e+04, 1.6808e+04, 2.2354e+04, 2.5139e+04, 3.5537e+04,
          5.0453e+04],
         [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+